In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### 解壓縮zip內容到colab RAM中, 再複製到drive空間內。注意不能指定到/content/drive/ 下面。好像會解壓縮不全。

In [ ]:
!unzip /content/drive/MyDrive/phm-ieee-2012-data-challenge-dataset-master.zip -d /content/

In [ ]:
!cp -r "/content/phm-ieee-2012-data-challenge-dataset-master" "/content/drive/MyDrive/TMP-for-DAST"

In [ ]:
# 下面的code可以觀察有沒有解壓csv成功
# import os
# _, _, files = next(os.walk("/content/drive/MyDrive/TMP-for-DAST/phm-ieee-2012-data-challenge-dataset-master/Full_Test_Set/Bearing2_4"))
# print(len(files))

#### 將完整的訓練與測試資料集搬到DAST-self/data/10FEMTO/raw_data/Validation_set下

In [ ]:
!mv /content/drive/MyDrive/TMP-for-DAST/phm-ieee-2012-data-challenge-dataset-master/Full_Test_Set/* /content/drive/MyDrive/DAST-self/data/10FEMTO/raw_data/Validation_set

In [ ]:
!mv /content/drive/MyDrive/TMP-for-DAST/phm-ieee-2012-data-challenge-dataset-master/Learning_set/* /content/drive/MyDrive/DAST-self/data/10FEMTO/raw_data/Validation_set

### 與訓練相關部分

In [6]:
%cd /content/drive/MyDrive/DAST-self/notebook/FEMTO-st

/content/drive/MyDrive/DAST-self/notebook/FEMTO-st


#### 預處理

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/DAST-self/notebook/FEMTO-st/data_processing')

In [4]:
from preprocess import Process
FEMTO_PATH = '../../data/10FEMTO/raw_data/Validation_set/'
POST_PROCESS_PATH = '../../data/10FEMTO/processed_data/'
# 想要快速驗證跑一個訓練一個測試即可
DATASET = [('Bearing1_2', True, 30), ('Bearing1_4', False, 50)]
# 跑全部資料集
# DATASET = [('Bearing1_1', True, 100), ('Bearing1_2', True, 30), ('Bearing1_3', False, 80), ('Bearing1_4', False, 50), ('Bearing1_5', False, 90), ('Bearing1_6', False, 90), ('Bearing1_7', False, 80),('Bearing2_1', True, 30), ('Bearing2_2', True, 30), ('Bearing2_3', False, 70), ('Bearing2_4', False, 27), ('Bearing2_5', False, 84), ('Bearing2_6', False, 25), ('Bearing2_7', False, 8), ('Bearing3_1', True, 17), ('Bearing3_2', True, 50), ('Bearing3_3', False, 15)]
WINDOW_SIZE = 40
MIN_SIGNAL_SIZE = 640
IS_SAVE = True

In [ ]:
process = Process(DATASET, WINDOW_SIZE, MIN_SIGNAL_SIZE, FEMTO_PATH, POST_PROCESS_PATH, IS_SAVE)
process.loop_folder()

#### DL

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/DAST-self/notebook/FEMTO-st/model_training')
sys.path.append('/content/drive/MyDrive/DAST-self/network')
from enums import NormType, TrainType
from train import DASTModel, DASTTF

In [18]:
selected_indices = [1, 3, 5, 7, 9, 10, 14]
# selected_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
FEATURE_LEN = len(selected_indices)
FEATURE_SIZE = 20
EMBEDD = 10
HYPER_PARAMETERS = {
    # model parameter
    'batch_size': 256,
    'dim_val': FEATURE_SIZE,
    'dim_attn': EMBEDD,
    'dim_val_t': FEATURE_SIZE,
    'dim_attn_t': EMBEDD,
    'dim_val_s': FEATURE_SIZE,
    'dim_attn_s': EMBEDD,
    'n_heads': 4,
    'n_decoder_layers': 1,
    'n_encoder_layers': 2,
    'lr': 1e-3,
    'epochs': 10,
    'time_step': 40,
    # limit how many last input used, important!
    'dec_seq_len': 6,
    'output_sequence_length': 1,
    'feature_len': FEATURE_LEN,
    'debug': True
    # else parameter
}

In [19]:
DATA_PATH = '../../data/10FEMTO/processed_data/'
# 快速驗證用
TRAIN_DATASETS = ['Bearing1_2']
TEST_DATASET = ['Bearing1_4']
# 自選哪個train, test資料集
# TRAIN_DATASETS = ['Bearing1_1', 'Bearing1_2']
# TEST_DATASET = ['Bearing1_3', 'Bearing1_4', 'Bearing1_5', 'Bearing1_6', 'Bearing1_7']
# TRAIN_DATASETS = ['Bearing2_1', 'Bearing2_2']
# TEST_DATASET = ['Bearing2_3', 'Bearing2_4', 'Bearing2_5', 'Bearing2_6', 'Bearing2_7']
# TRAIN_DATASETS = ['Bearing3_1', 'Bearing3_2']
# TEST_DATASET = ['Bearing3_3']

# 更改model儲存名稱
MODEL_SAVE_NAME = f'Bearing{TRAIN_DATASETS[0][7]}_pretrain_{len(selected_indices)}_colab_test'
NORM_TYPE = NormType.NO_NORM
TRAIN_TYPE = TrainType.DL
MODEL_SAVE_PATH = '../../model/'
IS_SAVE_MODEL = True

In [20]:
dast_model = DASTModel(train_datasets=TRAIN_DATASETS, test_dataset=TEST_DATASET, data_path=DATA_PATH, norm_type=NORM_TYPE, hyper_parameters=HYPER_PARAMETERS, model_save_path=MODEL_SAVE_PATH, model_save_name=MODEL_SAVE_NAME, is_save_model=IS_SAVE_MODEL, train_type=TRAIN_TYPE)

In [21]:
dast_model.main(selected_indices)

100%|██████████| 4/4 [00:02<00:00,  1.65it/s]


epoch =  0 train_loss =  0.3010791453998536
rmse_loss =  0.5870177745819092 mae_loss =  0.4598703384399414 mse_loss =  0.34458988904953003


100%|██████████| 4/4 [00:02<00:00,  1.58it/s]


epoch =  1 train_loss =  0.06269301008433104
rmse_loss =  0.5694174766540527 mae_loss =  0.5133672952651978 mse_loss =  0.3242362439632416


100%|██████████| 4/4 [00:03<00:00,  1.16it/s]


...

epoch =  7 train_loss =  0.03159363684244454
rmse_loss =  0.42596426606178284 mae_loss =  0.29240313172340393 mse_loss =  0.18144555389881134


100%|██████████| 4/4 [00:02<00:00,  1.69it/s]


epoch =  8 train_loss =  0.027284152107313275
rmse_loss =  0.40039390325546265 mae_loss =  0.305122971534729 mse_loss =  0.16031527519226074


100%|██████████| 4/4 [00:02<00:00,  1.45it/s]


epoch =  9 train_loss =  0.019625682849437
rmse_loss =  0.4388522505760193 mae_loss =  0.3735014796257019 mse_loss =  0.19259129464626312
train time: 0.2690176, s/epoch
embed1: 20, embed2: 20, lr: 0.001, dec_seq_len: 6
0.0196257
0.0871273
0.2951733


In [ ]:
dast_model.plt_loss_list()
dast_model.plt_best_predict()
dast_model.plt_last_predict_vs_y()

#### TL

In [3]:
selected_indices = [1, 3, 5, 7, 9, 10, 14]
# selected_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
FEATURE_LEN = len(selected_indices)
FEATURE_SIZE = 20
EMBEDD = 10
HYPER_PARAMETERS = {
    # model parameter
    'batch_size': 256,
    'dim_val': FEATURE_SIZE,
    'dim_attn': EMBEDD,
    'dim_val_t': FEATURE_SIZE,
    'dim_attn_t': EMBEDD,
    'dim_val_s': FEATURE_SIZE,
    'dim_attn_s': EMBEDD,
    'n_heads': 4,
    'n_decoder_layers': 1,
    'n_encoder_layers': 2,
    'lr': 1e-3,
    'epochs': 10,
    'time_step': 40,
    # limit how many last input used, important!
    'dec_seq_len': 6,
    'output_sequence_length': 1,
    'feature_len': FEATURE_LEN,
    'debug': True
}

In [4]:
DATA_PATH = '../../data/10FEMTO/processed_data/'
PRETRAIN_DATASET = 'Bearing2'
FINETUNE_DATASET = 'Bearing1'
# 快速驗證用
TRAIN_DATASETS = ['Bearing1_2']
TEST_DATASET = ['Bearing1_4']
# 自選哪個train, test資料集
# TRAIN_DATASETS = ['Bearing1_1', 'Bearing1_2']
# TEST_DATASET = ['Bearing1_3', 'Bearing1_4', 'Bearing1_5', 'Bearing1_6', 'Bearing1_7']
# TRAIN_DATASETS = ['Bearing2_1', 'Bearing2_2']
# TEST_DATASET = ['Bearing2_3', 'Bearing2_4', 'Bearing2_5', 'Bearing2_6', 'Bearing2_7']
# TRAIN_DATASETS = ['Bearing3_1', 'Bearing3_2']
# TEST_DATASET = ['Bearing3_3']

MODEL_SAVE_PATH = '../../model'
# 更改model儲存名稱
MODEL_SAVE_NAME = f'{PRETRAIN_DATASET}_pretrain_{FINETUNE_DATASET}_finetune_7_colab_test'
PRETRAIN_MODEL_PATH = '../../model'
PRETRAIN_MODEL_NAME = f'{PRETRAIN_DATASET}_pretrain_7'
IS_SAVE_MODEL = True
NORM_TYPE = NormType.NO_NORM
TRAIN_TYPE = TrainType.TL

In [7]:
pretrain_model = DASTTF(HYPER_PARAMETERS, PRETRAIN_MODEL_PATH, PRETRAIN_MODEL_NAME).pretrained_model

In [11]:
dast_model = DASTModel(train_datasets=TRAIN_DATASETS, test_dataset=TEST_DATASET, data_path=DATA_PATH, norm_type=NORM_TYPE, hyper_parameters=HYPER_PARAMETERS, model_save_path=MODEL_SAVE_PATH, model_save_name=MODEL_SAVE_NAME, is_save_model=IS_SAVE_MODEL, train_type=TRAIN_TYPE)
dast_model.set_model(pretrain_model)

In [ ]:
dast_model.main(selected_indices)

In [ ]:
dast_model.plt_loss_list()
dast_model.plt_best_predict()
dast_model.plt_last_predict_vs_y()